In [54]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import json
import os
from pathlib import Path
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

In [2]:
def getOrdersReturned(file_path):
    file_path = Path(file_path).resolve()

    try:
        with open(file_path, 'r') as jsonFile:
            jsonData = json.load(jsonFile)
        

        indexes = list(jsonData.keys())
        
        # Extracting column names
        jsonItems = list(jsonData.items())
        columns = list(jsonItems[0][1].keys())

        # Creating empty pandas dataframe
        order_returned_df = pd.DataFrame(columns=columns)

        for i in range(0, len(jsonItems)):
            itmeDict = dict(jsonItems[i][1].items())
            for key,item in itmeDict.items():
                order_returned_df.loc[i, key] = item
        

        return order_returned_df


        
    except Exception as e:
        print(e)

In [3]:
def getTransactionDetails(json_file_path):

    # Using pathlib.Path to handle file paths
    json_file_path = Path(json_file_path).resolve()

    try:
        with open(json_file_path, 'r') as file:
            data = json.load(file)
        
        # There are 3 fields in which data is stored in this json file: columns, index, data
        # We will extract column names and indexes in seperate lists and for data, we will prepare a list of lists.
        # It will help us to prepare a pandas dataframe to work with later on.

        column_names = data['columns']
        indexes = data['index']

        # Extracting all data rows
        dataObjets = data['data']
        dataRows = []

        for i in range(0, len(dataObjets)):
            dataRows.append(dataObjets[i])

        # Creating a pandas dataframe 
        customer_transaction_df = pd.DataFrame(dataRows, columns=column_names, index=indexes)

        return customer_transaction_df


    
    except Exception as e:
        pass
    except FileNotFoundError as e:
        print(e)
    
    
    

In [4]:
def getProductDetails(file_path):
    file_path = Path(file_path).resolve()

    try:
        with open(file_path, 'r') as jsonFile:
            data = json.load(jsonFile)
        
        schemaFields = data['schema']['fields']
        column_names = []

        dataObjects = data['data']

        for i in range(0, len(schemaFields)):
            column_names.append(schemaFields[i]['name'])

        product_df = pd.DataFrame(data=dataObjects, columns=column_names)

        return product_df

        

    except Exception as e:
        print(e)
    

In [5]:
def getCustomerInfo(file_path):

    file_path = Path(file_path).resolve()

    try:
        with open(file_path, 'r') as jsonFile:
            data = json.load(jsonFile)

        column_names = list(data[0].keys())
        dataObjects = []

        for i in range(0, len(data)):
            temp = []
            for keys in data[i]:
                temp.append(data[i][keys])
            
            dataObjects.append(temp)
        

        customer_info_df = pd.DataFrame(data=dataObjects, columns=column_names)
        return customer_info_df

    except Exception as e:
        print(e)

**Note: Currently, not focusing on region_seller_info**

#### Writing a function to clean customer_transaction dataset and also to merge it with orders_returned dataset

In [26]:

def preProcessProducts():
    """ 
    From initial observations,some of the productIDs are repeating in this dataset, althought the product names are different. Since, they're creating ambiguity in transactional information, we came up with following modifications:

    * Appended '-x' string at the end of each productID, where x is an integer. Initially, all 'unique' productIDs will have the x value to be 0.
    * As we encounter duplicate entries of same productID, we will go on incrmenting value of 'x' 
    * This will be our variantID. 

    The way transaction data is being handled, there's no room for duplicate productID entries to exist for same orderID and customerID combination.

    """
    product_info_df = getProductDetails(r'C:\SEM6\BMC_Hackathon\Datasets\TakeHome-Data Scientist\Data\product_info.json')

    product_info_df_sorted = product_info_df.sort_values(by="Product ID", ascending=True).reset_index(drop=True)

    def fixProductIDs():
        productID_count = {}
        modified_IDs = []

        for index, row in product_info_df_sorted.iterrows():
            # Get the corresponding productID for each row
            productID = row["Product ID"]

            if productID in productID_count:
                productID_count[productID] += 1
            else:
                productID_count[productID] = 0

            modified_IDs.append(f"{productID}-{productID_count[productID]}")            
        
        product_info_df_sorted["variant_id"] = modified_IDs
        

    

    fixProductIDs()

    product_info_df_sorted = product_info_df_sorted.rename(columns={'Product ID' : 'Old Product ID', 'variant_id': 'Product ID'})
    return product_info_df_sorted
        


    

    

In [ ]:
def preProcessTransactions(transactions_info_df: pd.DataFrame):
    """
    Some records were found to have same [OrderID, CustomerID, ProductID] combination with different values for Quantity
    To resolve this issue, we decided to aggregate these differing features such as Profit, Quantity bought and Sales amount by summing them up

    """
    try:
        columns_to_check = [
            "Order ID",
            "Order Date",
            "Ship Date",
            "Ship Mode",
            "Customer ID",
            "Product ID",
            "Discount",
        ]
        temp_merged_quantity_df = transactions_info_df.groupby(
            columns_to_check, as_index=False
        ).agg({"Quantity": "sum", "Sales": "sum", "Profit": "sum"})

        transactions_info_df = temp_merged_quantity_df

        """ 
        2] The order and shipping dates are in UNIX timestamp format. During model training, how to use these dates becomes a question. Instead, we will be converting these dates to the ususal date-time format, and extracting the no of days have passed since the particular item was last bought

        This could prove to be crucial for predictive analytics.

        """

        current_date = datetime.today().date()
        transactions_info_df["Order_date_modified"] = transactions_info_df[
            "Order Date"
        ].apply(lambda x: pd.to_datetime(x, unit="ms").date())

        transactions_info_df["last_ordered"] = transactions_info_df[
            "Order_date_modified"
        ].apply(lambda x: (current_date - x).days)

        transactions_info_df = transactions_info_df.drop(
            columns=["Order_date_modified", "Order Date", "Ship Date"]
        )

        """ 
        Reading the orders_returned data into a dataframe.

        """
        orders_returned_df = getOrdersReturned(
            r"C:\SEM6\BMC_Hackathon\Datasets\TakeHome-Data Scientist\Data\orders_returned_info.json"
        )

        temp_merged_df = transactions_info_df.merge(
            orders_returned_df, on="Order ID", how="outer"
        )

        temp_merged_df["Returned"] = temp_merged_df["Returned"].fillna("No")
        transactions_info_df = temp_merged_df


        """ 
        After we have created a variantID for each repeating productID, we need to make subsequent changes for productIDs in the transactions table too.
        * Since there's no way as of now to understand the business logic behind the which product was bought from the products falling under the same productID, the idea is to replace it with first occurence of variantID in the product Info table
    
        """
        product_df = preProcessProducts()
        variant_mappings = product_df.groupby("Old Product ID")["Product ID"].first().to_dict()
        transactions_info_df["Product ID"] = transactions_info_df["Product ID"].map(variant_mappings).fillna(transactions_info_df["Product ID"])

        # Performing left join on transaction data with the product information

        temp_joined_df = transactions_info_df.merge(product_df, on="Product ID", how="inner")
        temp_joined_df = temp_joined_df.drop(columns=["Old Product ID", "index"])


        # Merging customer data with transactions
        # customer_df = getCustomerInfo(r"C:\SEM6\BMC_Hackathon\Datasets\TakeHome-Data Scientist\Data\customers_info.json")
        # temp_joined_df = temp_joined_df.merge(customer_df, on="Customer ID", how="inner")
        # temp_joined_df = temp_joined_df.drop(columns=["Country", "City", "State", "Region"])

        
        transactions_info_df = temp_joined_df
        

        return transactions_info_df
    except Exception as e:
        print(e)

### ****END OF PRE-PROCESSING****

In [31]:
transaction_df = getTransactionDetails(r'C:\SEM6\BMC_Hackathon\Datasets\TakeHome-Data Scientist\Data\customer_transaction_info.json')
transaction_df

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Product ID,Sales,Quantity,Discount,Profit
0,CA-2016-152156,1478563200000,1478822400000,Second Class,CG-12520,FUR-BO-10001798,261.9600,2,0.00,41.9136
1,CA-2016-152156,1478563200000,1478822400000,Second Class,CG-12520,FUR-CH-10000454,731.9400,3,0.00,219.5820
2,CA-2016-138688,1465689600000,1466035200000,Second Class,DV-13045,OFF-LA-10000240,14.6200,2,0.00,6.8714
3,US-2015-108966,1444521600000,1445126400000,Standard Class,SO-20335,FUR-TA-10000577,957.5775,5,0.45,-383.0310
4,US-2015-108966,1444521600000,1445126400000,Standard Class,SO-20335,OFF-ST-10000760,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...
9989,CA-2014-110422,1390262400000,1390435200000,Second Class,TB-21400,FUR-FU-10001889,25.2480,3,0.20,4.1028
9990,CA-2017-121258,1488067200000,1488499200000,Standard Class,DB-13060,FUR-FU-10000747,91.9600,2,0.00,15.6332
9991,CA-2017-121258,1488067200000,1488499200000,Standard Class,DB-13060,TEC-PH-10003645,258.5760,2,0.20,19.3932
9992,CA-2017-121258,1488067200000,1488499200000,Standard Class,DB-13060,OFF-PA-10004041,29.6000,4,0.00,13.3200


In [93]:
transaction_df_1 = preProcessTransactions(transactions_info_df=transaction_df)
transaction_df_1.head()

,Order ID,Ship Mode,Customer ID,Product ID,Discount,Quantity,Sales,Profit,last_ordered,Returned,Category,Sub-Category,Product Name
0,CA-2014-100006,Standard Class,DK-13375,TEC-PH-10002075-0,0.0,3,377.970,109.6113,3822,No,Technology,Phones,AT&T EL51110 DECT
1,CA-2014-100090,Standard Class,EB-13705,FUR-TA-10003715-0,0.2,3,502.488,-87.9354,3883,No,Furniture,Tables,Hon 2111 Invitation Series Corner Table
2,CA-2014-100090,Standard Class,EB-13705,OFF-BI-10001597-0,0.2,6,196.704,68.8464,3883,No,Office Supplies,Binders,"Wilson Jones Ledger-Size, Piano-Hinge Binder, ..."
3,CA-2014-100293,Standard Class,NF-18475,OFF-PA-10000176-0,0.2,6,91.056,31.8696,3999,No,Office Supplies,Paper,Xerox 1887
4,CA-2014-100328,Standard Class,JC-15340,OFF-BI-10000343-0,0.2,1,3.928,1.3257,4044,No,Office Supplies,Binders,"Pressboard Covers with Storage Hooks, 9 1/2"" x..."


#### For predictive modelling, these features cannot be use in a straightforward manner.  
#### We need to perform feature engineering, to extract product, customer and transaction oriented features.

In [86]:
customer_pincode_grouped = transaction_df_1.groupby(["Customer ID", "Product ID", "Postal Code"]).agg(
    total_orders = ("Order ID", "count"),
    total_sales= ("Sales", "sum"),
    avg_order_value=("Sales", "mean"),
    total_quantity=("Quantity", "sum"),
    avg_discount=("Discount", "mean"),
    return_rate=("Returned", lambda x: (x=="Yes").mean()),
    last_ordered=("last_ordered", "min")
).reset_index()

In [87]:
customer_pincode_grouped

,Customer ID,Product ID,Postal Code,total_orders,total_sales,avg_order_value,total_quantity,avg_discount,return_rate,last_ordered
0,AA-10315,FUR-FU-10002456-0,10011,1,14.56,14.56,2,0.0,0.0,3814
1,AA-10315,FUR-FU-10002456-0,55407,1,14.56,14.56,2,0.0,0.0,3814
2,AA-10315,FUR-FU-10002456-0,78664,1,14.56,14.56,2,0.0,0.0,3814
3,AA-10315,FUR-FU-10002456-0,94109,1,14.56,14.56,2,0.0,0.0,3814
4,AA-10315,FUR-FU-10002456-0,94122,1,14.56,14.56,2,0.0,0.0,3814
...,...,...,...,...,...,...,...,...,...,...
71498,ZD-21925,TEC-PH-10002824-0,32216,1,823.96,823.96,5,0.2,0.0,3152
71499,ZD-21925,TEC-PH-10002824-0,37421,1,823.96,823.96,5,0.2,0.0,3152
71500,ZD-21925,TEC-PH-10002824-0,47374,1,823.96,823.96,5,0.2,0.0,3152
71501,ZD-21925,TEC-PH-10002824-0,90036,1,823.96,823.96,5,0.2,0.0,3152


In [91]:
transaction_df_1[(transaction_df_1["Customer ID"] == "ZD-21925") & (transaction_df_1["Product ID"] == "TEC-PH-10002824-0")]

,Order ID,Ship Mode,Customer ID,Product ID,Discount,Quantity,Sales,Profit,last_ordered,Returned,Category,Sub-Category,Product Name,Customer Name,Segment,Postal Code
36076,CA-2016-152471,Same Day,ZD-21925,TEC-PH-10002824-0,0.2,5,823.96,51.4975,3152,No,Technology,Phones,Jabra SPEAK 410 Multidevice Speakerphone,Zuschuss Donatelli,Consumer,94109
36077,CA-2016-152471,Same Day,ZD-21925,TEC-PH-10002824-0,0.2,5,823.96,51.4975,3152,No,Technology,Phones,Jabra SPEAK 410 Multidevice Speakerphone,Zuschuss Donatelli,Consumer,37421
36078,CA-2016-152471,Same Day,ZD-21925,TEC-PH-10002824-0,0.2,5,823.96,51.4975,3152,No,Technology,Phones,Jabra SPEAK 410 Multidevice Speakerphone,Zuschuss Donatelli,Consumer,32216
36079,CA-2016-152471,Same Day,ZD-21925,TEC-PH-10002824-0,0.2,5,823.96,51.4975,3152,No,Technology,Phones,Jabra SPEAK 410 Multidevice Speakerphone,Zuschuss Donatelli,Consumer,47374
36080,CA-2016-152471,Same Day,ZD-21925,TEC-PH-10002824-0,0.2,5,823.96,51.4975,3152,No,Technology,Phones,Jabra SPEAK 410 Multidevice Speakerphone,Zuschuss Donatelli,Consumer,90036


In [88]:
transaction_df_1_merged = transaction_df_1.merge(customer_pincode_grouped, on=["Customer ID", "Product ID", "Postal Code"], how="inner")

In [ ]:
label_encoders = {}
categorical_columns = ["Ship Mode", "Segment", "Category", "Sub-Category" ]
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    transaction_df_1[col] = label_encoders[col].fit_transform(transaction_df_1[col])



In [57]:
product_label_enc = LabelEncoder()
transaction_df_1["Product ID"] = product_label_enc.fit_transform(transaction_df_1["Product ID"])

In [58]:
transaction_df_1

,Order ID,Ship Mode,Customer ID,Product ID,Discount,Quantity,Sales,Profit,last_ordered,Returned,Category,Sub-Category,Product Name,Customer Name,Segment,Postal Code
0,CA-2014-100006,3,DK-13375,1752,0.0,3,377.970,109.6113,3822,No,2,13,AT&T EL51110 DECT,Dennis Kane,0,43302
1,CA-2014-100006,3,DK-13375,1752,0.0,3,377.970,109.6113,3822,No,2,13,AT&T EL51110 DECT,Dennis Kane,0,92105
2,CA-2014-100006,3,DK-13375,1752,0.0,3,377.970,109.6113,3822,No,2,13,AT&T EL51110 DECT,Dennis Kane,0,10024
3,CA-2014-100006,3,DK-13375,1752,0.0,3,377.970,109.6113,3822,No,2,13,AT&T EL51110 DECT,Dennis Kane,0,48185
4,CA-2014-100006,3,DK-13375,1752,0.0,3,377.970,109.6113,3822,No,2,13,AT&T EL51110 DECT,Dennis Kane,0,98105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71873,US-2017-169551,0,RL-19615,1720,0.4,2,683.988,-113.9980,2788,No,2,13,Apple iPhone 5S,Rob Lucas,0,19120
71874,US-2017-169551,0,RL-19615,1720,0.4,2,683.988,-113.9980,2788,No,2,13,Apple iPhone 5S,Rob Lucas,0,60623
71875,US-2017-169551,0,RL-19615,1720,0.4,2,683.988,-113.9980,2788,No,2,13,Apple iPhone 5S,Rob Lucas,0,90049
71876,US-2017-169551,0,RL-19615,1720,0.4,2,683.988,-113.9980,2788,No,2,13,Apple iPhone 5S,Rob Lucas,0,90045
